In [4]:
import sys
sys.path.append("../")
from aoc_utils import *

In [5]:
from aocd import get_data, submit

def get_in_file(day):
    with open( f"input/input{day}.txt", "w") as f:
        f.write( get_data(day=day, year=2021) )

Each day's work will consist of three tasks:
- **Input**: Parse the day's input file with the function `parse(day, parser, sep)`, which treats the input as a sequence of *entries*, separated by `sep` (default newline); applies `parser` to each entry; and returns the results as a tuple. (Note: `ints` and `atoms` are useful `parser` functions (as are `int` and `str`).)
- **Part 1**: Write code to compute the answer to Part 1, and submit the answer to the AoC site. Use the function `answer` to record the correct answer and serve as a regression test when I re-run the notebook.
- **Part 2**: Repeat coding and `answer` for Part 2.


In [6]:
def parse(day, parser=str, sep='\n') -> tuple:
    """Split the day's input file into entries separated by `sep`, and apply `parser` to each."""
    entries = open(f'input/input{day}.txt').read().rstrip().split(sep)
    return mapt(parser, entries)

# [Day 1](https://adventofcode.com/2021/day/1): Sonar Sweep

count the number of times a depth measurement increases from the previous measurement

In [ ]:
get_in_file(1)
in1 = parse(1, int)
incr = Counter([1 if b > a else -1 for a, b in pairwise(in1)])[1] 
submit(incr, part="a", day=1, year=2021)

### Part Two

consider sums of a three-measurement sliding window

In [ ]:
sw = [c+b+a for a,b,c in sliding_window(in1, 3)] 
incr = Counter([1 if b > a else -1 for a,b in pairwise(sw)])[1]
submit(incr, part="b", day=1, year=2021)

# [Day 2](https://adventofcode.com/2021/day/2): Dive!

- forward X increases the horizontal position by X units.
- down X increases the depth by X units.
- up X decreases the depth by X units.

In [ ]:
get_in_file(2)
parse_in = lambda x : { k:int(v) for (k,v) in [str.split(x)] }
in2 = Input(2, parse_in)

In [ ]:
c = Counter()
[c.update(d) for d in in2]; c

In [ ]:
course = c['forward']*(c['down']-c['up'])
submit(course, part="a", day=2, year=2021)

### Part Two

- down X increases your aim by X units.
- up X decreases your aim by X units.
- forward X does two things:
    - It increases your horizontal position by X units.
    - It increases your depth by your aim multiplied by X.

In [ ]:
aim = 0
horiz = 0
depth = 0
for d in in2 :
    for k,v in d.items() :
        if k == 'down':
            aim += v
        if k == 'up':
            aim -= v
        if k == 'forward':
            horiz += v
            depth += aim * v

aim, horiz, depth
submit(horiz*depth, part="b", day=2, year=2021)

# [Day 3](https://adventofcode.com/2021/day/3): Binary Diagnostic

Each bit in the gamma rate can be determined by finding the most common bit in the corresponding position of all numbers in the diagnostic report

The epsilon rate is calculated in a similar way; rather than use the most common bit, the least common bit from each position is used

The power consumption can then be found by multiplying the gamma rate by the epsilon rate.

In [ ]:
get_in_file(3)
parse_in = lambda x : mapt(int, list(str.strip(x)))
in3 = Input(3, parse_in)

In [ ]:
inT = np.array(in3).T
print(f"input Transposed shape {inT.shape}")
bits_counter = mapt( lambda x : Counter(x.tolist()).most_common(), inT ); bits_counter

In [ ]:
gamma_rate_bits = mapt( lambda x : Counter(x.tolist()).most_common()[0][0], inT )
print (f"gamma_rate_bits {gamma_rate_bits}")
gamma_rate = int("".join(map(str,gamma_rate_bits)), 2); gamma_rate

In [ ]:
epsilon_rate_bits = mapt( lambda x : Counter(x.tolist()).most_common()[-1][0], inT )
print (f"epsilon_rate_bits {epsilon_rate_bits}")
epsilon_rate = int("".join(map(str,epsilon_rate_bits)), 2); epsilon_rate

In [ ]:
submit(gamma_rate*epsilon_rate, part="a", day=3, year=2021)

### Part Two

The bit criteria depends on which type of rating value you want to find:

- To find oxygen generator rating, determine the most common value (0 or 1) in the current bit position, and keep only numbers with that bit in that position. If 0 and 1 are equally common, keep values with a 1 in the position being considered.
- To find CO2 scrubber rating, determine the least common value (0 or 1) in the current bit position, and keep only numbers with that bit in that position. If 0 and 1 are equally common, keep values with a 0 in the position being considered.

In [ ]:

def Oxy_bit_criteria(x) :
    c = Counter(x.tolist())
    #print (c)
    if c[0] == c[1]:
        return 1
    return c.most_common()[0][0]

def CO2_bit_criteria(x) :
    c = Counter(x.tolist())
    #print (c)
    if c[0] == c[1]:
        return 0
    return c.most_common()[-1][0]

def calc_rate(input, bit_fun_criteria) :

    col = 0
    inM = np.array(input)

    while ( inM.shape[0] > 1 ) :
        # find criteria bit in column "col"
        criteria_bit = mapt( bit_fun_criteria, inM.T )[col]
        #print (f"col {col} criteria bit {criteria_bit}")
        # get a boolean vector which select rows that have "most_common_bit" at column "col" 
        row_selector = inM[:,col] == criteria_bit
        # filter all the rows using boolean vector "row_selector" 
        inM = inM[row_selector,:]
        col += 1

    return inM[0]

In [ ]:
oxygen_generator_rating = int("".join(map(str, calc_rate(in3, Oxy_bit_criteria))), 2); oxygen_generator_rating

In [ ]:
co2_scrubber_rating = int("".join(map(str, calc_rate(in3, CO2_bit_criteria))), 2); co2_scrubber_rating

In [ ]:
submit(oxygen_generator_rating*co2_scrubber_rating, part="b", day=3, year=2021)

# [Day 4](https://adventofcode.com/2021/day/4): Giant Squid

Bingo is played on a set of boards each consisting of a 5x5 grid of numbers. Numbers are chosen at random, and the chosen number is marked on all boards on which it appears. (Numbers may not appear on all boards.) If all numbers in any row or any column of a board are marked, that board wins. (Diagonals don't count.)

In [ ]:
get_in_file(4)

in4 = parse(4, ints, sep="\n\n")

def bingo_score(input) :
    order, *boards = input
    boardsM = mapt(lambda b: np.array(b).reshape(5,5), boards)
    boardsK = mapt(lambda b: np.full((5,5), False), boards)
    it_order = iter(order)
    tmpK = list()
    drawn = list()
    bingo = False

    while not bingo :
        draw = next(it_order)
        for bM,bK in zip(boardsM, boardsK) :
            # mask elem is True where "draw" is in bM
            mask = np.isin(bM, draw)
            bK = bK | mask
            # If all numbers in any row or any column of a board are marked
            # that board wins
            if ( np.any(np.all(bK, axis=0)) or 
                 np.any(np.all(bK, axis=1)) ) :
                bingo = True
                break
            
            tmpK.append(bK)
        #
        drawn.append(draw)
        boardsK = tmpK
        tmpK = list()

    return (draw,drawn,bM,bK)

d,D,M,K = bingo_score(in4)
print (M)
#print (D)
#print (K)
#print (np.logical_not(K))
score = np.sum(M[np.logical_not(K)])*d


In [ ]:
submit(score, part="a", day=4, year=2021)

### Part Two

... figure out which board will win last

In [ ]:
def win_boards(input) :
    order, *boards = input
    boardsM = mapt(lambda b: np.array(b).reshape(5,5), boards)
    boardsK = mapt(lambda b: np.full((5,5), False), boards)
    it_order = iter(order)
    tmpK = list()
    tmpM = list()
    drawn = list()
    bingo = False
    bingo_boards = list()

    while True :
        try :
            draw = next(it_order)
            for bM,bK in zip(boardsM, boardsK) :
                # mask elem is True where "draw" is in bM
                mask = np.isin(bM, draw)
                bK = bK | mask
                # If all numbers in any row or any column of a board are marked
                # that board wins
                if ( np.any(np.all(bK, axis=0)) or 
                    np.any(np.all(bK, axis=1)) ) :
                    bingo_boards.append((draw,bM,bK))
                else:
                    tmpK.append(bK)
                    tmpM.append(bM)
            #
            drawn.append(draw)
            boardsK = tmpK
            boardsM = tmpM
            tmpK = list()
            tmpM = list()
        except (StopIteration) :
            break

    return bingo_boards

d,M,K = win_boards(in4)[-1]

score = np.sum(M[np.logical_not(K)])*d

In [ ]:
submit(score, part="b", day=4, year=2021)

# [Day 5](https://adventofcode.com/2021/day/5): Hydrothermal Venture

In [ ]:
get_in_file(5)

in5 = parse(5, ints)

In [ ]:
class Point(namedtuple('Point', ['x', 'y'])):
    pass
"""
    def __eq__(self, other):
        return ((self.x, self.y) == (other.x, other.y))
    def __ne__(self, other):
        return not (self == other)
    def __lt__(self, other):
        return ((self.last, self.first) < (other.last, other.first))
"""    

class Segment(namedtuple('Segment', ['p1', 'p2'])):
    def is_V(self):
        return self.p1.x == self.p2.x
    def is_H(self):
        return self.p1.y == self.p2.y
    def is_D(self):
        return (abs(self.p1.x - self.p2.x) == abs(self.p1.y - self.p2.y))
    def min_x(self):
        return min(self.p1.x ,self.p2.x)
    def min_y(self):
        return min(self.p1.y ,self.p2.y)
    def max_x(self):
        return max(self.p1.x ,self.p2.x)
    def max_y(self):
        return max(self.p1.y ,self.p2.y)
    def slope(self):
        return (self.p2.y - self.p1.y)/(self.p2.x - self.p1.x)
    def get_points(self):
        if ( self.is_V() ) :
            return mapt(lambda y: Point(self.p1.x,y), range(self.min_y(),self.max_y()+1))
        if ( self.is_H() ) :
            return mapt(lambda x: Point(x,self.p1.y), range(self.min_x(),self.max_x()+1))
        if ( self.is_D() ) :
            if self.slope() > 0 :
                return mapt(lambda z: Point(z[0],z[1]),
                            zip(range(self.min_x(),self.max_x()+1),
                                range(self.min_y(),self.max_y()+1)))
            else :
                return mapt(lambda z: Point(z[0],z[1]),
                            zip(range(self.max_x(),self.min_x()-1, -1),
                                range(self.min_y(),self.max_y()+1)))


In [ ]:
lines = [Segment(Point(x1,x2),Point(y1,y2)) for (x1,x2,y1,y2) in in5]
HV_lines = [line for line in lines if line.is_H() or line.is_V()]

In [ ]:
maxx = max([seg.max_x() for seg in HV_lines])
maxy = max([seg.max_y() for seg in HV_lines])
maxx,maxy

In [ ]:
pts_cnt = Counter()
mapt(lambda x: pts_cnt.update(Counter(x.get_points())), HV_lines)
# ... number of points where at least two lines overlap ==> >= 2
pts_num = len([v for v in pts_cnt.values() if v > 1])
pts_num = len([v for k,v in pts_cnt.most_common() if v > 1])
pts_num

In [ ]:
submit(pts_num, part="a", day=5, year=2021)

### Part Two

In [ ]:
test1 = [Segment(Point(1,1),Point(3,3)),Segment(Point(4,4),Point(2,2)),
         Segment(Point(0,1),Point(3,4)),Segment(Point(2,3),Point(0,1)),
         Segment(Point(9,7),Point(7,9))]
for t in test1 :
    t.is_D()
    print (t.slope(), t.get_points())


In [ ]:
D_lines = [line for line in lines if line.is_D()]
pts_cnt = Counter()
mapt(lambda x: pts_cnt.update(Counter(x.get_points())), chain(HV_lines,D_lines))
pts_num = len([v for k,v in pts_cnt.most_common() if v > 1])
pts_num

In [ ]:
submit(pts_num, part="b", day=5, year=2021)

# [Day 6](https://adventofcode.com/2021/day/6): Lanternfish


In [7]:
get_in_file(6)
in6 = parse(6, ints)

In [8]:
init_state, = in6
init_test = 3,4,3,1,2

In [9]:
def population (in_s, days) :
    D = deque()
    curr_state = in_s 
    while days > 0 :
        print (f"...days {days} len(curr_state)={curr_state}")
        c = Counter(curr_state)
        next_state = list(map(lambda x: 6 if x == 0 else x-1 , curr_state))
        next_state += c[0]*[8]
        #D.append(next_state)
        days -= 1
        curr_state = next_state
    return next_state

In [11]:
days = 80;  print (f"after {days} days {len(population(init_test, days))}" )
#days = 256; print (f"after {days} days {len(population(init_test, days))}" )

...days 80
...days 79
...days 78
...days 77
...days 76
...days 75
...days 74
...days 73
...days 72
...days 71
...days 70
...days 69
...days 68
...days 67
...days 66
...days 65
...days 64
...days 63
...days 62
...days 61
...days 60
...days 59
...days 58
...days 57
...days 56
...days 55
...days 54
...days 53
...days 52
...days 51
...days 50
...days 49
...days 48
...days 47
...days 46
...days 45
...days 44
...days 43
...days 42
...days 41
...days 40
...days 39
...days 38
...days 37
...days 36
...days 35
...days 34
...days 33
...days 32
...days 31
...days 30
...days 29
...days 28
...days 27
...days 26
...days 25
...days 24
...days 23
...days 22
...days 21
...days 20
...days 19
...days 18
...days 17
...days 16
...days 15
...days 14
...days 13
...days 12
...days 11
...days 10
...days 9
...days 8
...days 7
...days 6
...days 5
...days 4
...days 3
...days 2
...days 1
after 80 days 5934


In [14]:
state = population(init_state,80)
submit(len(state), part="a", day=6, year=2021)

Part a already solved with same answer: 353079


### Part Two

How many lanternfish would there be after 256 days?